## Text mining en Collectieve Intelligentie 

### Vandaag

1. Overzicht cursus en cursus website
3. NLTK chapter 3
2. More programming with text
    1. Regular expressions
3. Plotting 

### Literature
* PH 5, alleen deel over  _Automatic Readability Index_
* NLTK 3
* PyPlot:  <http://matplotlib.org/users/pyplot_tutorial.html>

# NLTK chapter 3: Processing raw text

## Main Topics

* Reading documents from the web
* String processing
* Regular expressions
* Encodings
* Text normalization,
* Text tokenization
* Text segmentation


# Blended learning

* We wisselen _hoorcollege_ en _werkcollege_ af.
* **Zorg dus dat je een notebook heb draaien**
* En zet deze slides open in je notebook.
* Dan kan je makkelijk de kleine quizjes proberen.

* <http://maartenmarx.nl/teaching/CollectieveIntelligentie/Slides/slides_week_2.ipynb>

# Reading documents from the web

### Most used packages

* Requests <http://docs.python-requests.org/en/latest/>
    * Helps you downloading pages
    * Good alternative for the often used `urllib2` <http://www.pythonforbeginners.com/python-on-the-web/how-to-use-urllib2-in-python/>
* beautifulsoup4 <http://www.pythonforbeginners.com/python-on-the-web/beautifulsoup-4-python/>
    * For parsing pages



In [2]:
import requests
from bs4 import BeautifulSoup
import re

In [3]:
url = "http://www.pythonforbeginners.com"

# download the url
r = requests.get(url)

# parse the page
soup = BeautifulSoup(r.text)

# Now you can do many things:
print soup.prettify()[:200]

 
 

<!DOCTYPE html>
<html lang="en-US" prefix="og: http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width" name="viewport"/>
  <title>
   Pythonforbeginners.com - Learn 


In [5]:
print soup.title.string
 

 Pythonforbeginners.com - Learn Python by Example 


In [4]:
# Find all links
links = soup.findAll('a')
print len(links)

links[:5]

87


[<a class="navbar-brand" href="/" title="pythonforbeginners.com">
 <img alt="pythonforbeginners.com" src="http://s.pythonforbeginners.com/static/img/pfb_logo.png"/>
 </a>,
 <a href="/" title="Home">Home</a>,
 <a href="/learn-python/" title="Learn Python">Learn Python</a>,
 <a href="/basics/" title="Basics">Basics</a>,
 <a href="/lists/" title="lists">Lists</a>]

In [6]:
# Put all titles of the links into a list

LinkTitles = [ a.get('title') for a in links]
LinkTitles[:5] 

['pythonforbeginners.com', 'Basics', 'lists', 'dictionary', 'code snippets']

In [5]:
# Make a list of all "anchor text"
Anchortext = [a.text for a in links]
print Anchortext

[u'\n\n', u'Home', u'Learn Python', u'Basics', u'Lists', u'Dictionary', u'Code Snippets', u'Modules', u'The del Statement', u'', u'\n                        read more \u2192', u'\xa0\xa0Basics', u'__str__ vs. __repr__', u'', u'\n                        read more \u2192', u'\xa0\xa0Basics', u'Break and Continue Statements', u'', u'\n                        read more \u2192', u'\xa0\xa0Basics', u'\xa0\xa0Code', u'Numeric Types in Python', u'', u'\n                        read more \u2192', u'\xa0\xa0Code', u'DNS Lookup With Python', u'', u'\n                        read more \u2192', u'\xa0\xa0Python', u'\xa0\xa0Sockets', u'Encoding JSON with Python', u'', u'\n                        read more \u2192', u'\xa0\xa0Dictionary', u'\xa0\xa0Json', u'\xa0\xa0Python', u'How to Use MySQL Connector/Python', u'', u'\n                        read more \u2192', u'\xa0\xa0MySQL', u'\xa0\xa0Pip', u'\xa0\xa0Python', u'PostGen', u'', u'\n                        read more \u2192', u'\xa0\xa0Basics', u'Get

## Quiz: your turn
1. Create a list containing triples, of (anchor text, url, title) for each a-element 

```
LinkTitles = [ a.get('title') for a in links]
Anchortext = [a.text for a in links]
```

#### Update 2016: this is not that relevant anymore for this specific page
2. That text in `Anchortext` looks awful. Use string methods to clean it and remove unwanted parts (like multiple spaces, newlines, etc.)

TIP: Want to know which string-methods exist? Make a test string `a='a'`, run it, and then ask `a.TAB` 

3. Use the Counter method to count the strings in anchor text
```
from collections import Counter
```




In [17]:
# Create a list containing triples, of (anchor text, url, title) for each a-element 
triples = [(a.get('href'),a.get('title'),a.text)  for a in links ]

triples[:11]

[('/', 'pythonforbeginners.com', u'\n\n'),
 ('/', 'Home', u'Home'),
 ('/learn-python/', 'Learn Python', u'Learn Python'),
 ('/basics/', 'Basics', u'Basics'),
 ('/lists/', 'lists', u'Lists'),
 ('/dictionary/', 'dictionary', u'Dictionary'),
 ('/code-snippets-source-code/', 'code snippets', u'Code Snippets'),
 ('/modules-in-python/', 'modules', u'Modules'),
 ('/basics/del-statement', 'The del Statement', u'The del Statement'),
 ('del-statement#disqus_thread', 'Comment on The del Statement', u''),
 ('/basics/del-statement',
  'The del Statement',
  u'\n                        read more \u2192')]

In [20]:
Anchortext[10]

u'\n                        read more \u2192'

In [21]:
Anchortext[10].strip()

u'read more \u2192'

In [16]:
#That text in `Anchortext` looks awful. 
# Use string methods to clean it and remove unwanted parts (like multiple spaces, newlines, etc.)
StrippedAnchors=[a.strip() for a in Anchortext]

for i in range(11):
    print Anchortext[i],'===>', StrippedAnchors[i]
 



===> 
Home ===> Home
Learn Python ===> Learn Python
Basics ===> Basics
Lists ===> Lists
Dictionary ===> Dictionary
Code Snippets ===> Code Snippets
Modules ===> Modules
The del Statement ===> The del Statement
 ===> 

                        read more → ===> read more →


In [23]:
#Use the Counter method to count the strings in anchor text
from collections import  Counter

AnchorCounts= Counter([a.lower() for a in StrippedAnchors])  # We also lower case
AnchorCounts.most_common(10)

[(u'', 11),
 (u'read more \u2192', 10),
 (u'basics', 6),
 (u'code snippets', 3),
 (u'dictionary', 3),
 (u'python', 3),
 (u'code', 2),
 (u'pip', 2),
 (u'lists', 2),
 (u'modules', 2)]

## Wrap up

1. BeautifulSoup is very powerful.
1. Can parse almost all HTML, also really bad, broken HTML.
    * _Bad broken HTML_????
    

1. For well-formed XML, you can use other parsers.
1. `import feedparser` for RSS feeds
1. `import lxml` for XML

# String processing

* Strings share many properties with lists
    * concatenate with `+`
    * slicing `kkk'[1:]`
    * `len`
    * `for c in 'hallo':`
    * _but they are immutable, where lists are not!_
        * `list[0]=1` is OK, but `'strong'[3]='i'` is not possible.
* and have many own methods: `string.TAB` in IPython
    * tests `islower()`
    * changers: `strip()`, `lower()`, `S.replace(old, new[, count]) -> string`

* but for harder tasks we need stronger machinery

### QUIZ: 
Create a list of  all words ending in 'en' from this list of words.
```
L=['stoel','stoelen','Ik' ,'ben', 'aan het', 'lopen']
```



In [6]:
L=['stoel','stoelen','Ik' ,'ben', 'aan het', 'lopen', 'enne', 'enkele', 'denne']

Len=[w for w in L if w[-2:]=='en']
Len

['stoelen', 'ben', 'lopen']

In [13]:
import re
Len=[w for w in L if  re.search(r'en$',w)]
Len

['stoelen', 'ben', 'lopen']

In [14]:
# Forget the dollar
Len=[w for w in L if  re.search(r'en',w)]
Len

['stoelen', 'ben', 'lopen', 'enne', 'enkele', 'denne']

# Regular expressions

[Own set of slides](PythonRegex.ipynb)

#Further text processing:

* Text normalization,
* Text tokenization
* Text segmentation

# Text normalization

* Idea: we are most often not interested in the _string_ but in the _concept_ behind it.
* But many ways to write the same concept:
    * stoel, stoelen, Stoel, Stoeltje, sTOEletje, $\ldots$
    * Proper names
    * verb and other conjugations

## Examples
* lower casing is almost always OK (without loss of meaning)
* lemmatizing
    * reduce wordforms to their stem/lemma
    * expensive as it needs dictionary lookup for each word
    * stemming: rule based heuristic approach to bring words back to their stem
        * more false hits then with lemmatization.
* Lemmatization example
    * _Ik werk hard aan mijn werk._
   

## Segmentation

* Cluster words into larger meaningful units:
    * sentence
    * paragraph
    * $\ldots$ 
* often application dependent

### Next week more

In [7]:
!ipython nbconvert slides_week_2.ipynb    --to slides --reveal-prefix "http://maartenmarx.nl/reveal.js" 
!mv  slides_week_2.slides.html slides_week_2.html
!chmod a+r slides_week_2*

[NbConvertApp] Using existing profile dir: u'/Users/admin/.ipython/profile_default'
[NbConvertApp] Converting notebook slides_week_2.ipynb to slides
[NbConvertApp] Support files will be in slides_week_2_files/
[NbConvertApp] Loaded template slides_reveal.tpl
[NbConvertApp] Writing 229595 bytes to slides_week_2.slides.html
